In [5]:
#!/usr/bin/env python2
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals
import os, sys
import numpy as np
import json
import time
from datetime import timedelta
from collections import defaultdict
import argparse
import multiprocessing

import PIL.Image as Image

from utils import get_traceback, rgb2id

In [2]:
class PQStatCat():
        def __init__(self):
            self.iou = 0.0
            self.tp = 0
            self.fp = 0
            self.fn = 0

        def __iadd__(self, pq_stat_cat):
            self.iou += pq_stat_cat.iou
            self.tp += pq_stat_cat.tp
            self.fp += pq_stat_cat.fp
            self.fn += pq_stat_cat.fn
            return self


class PQStat():
    def __init__(self):
        #Creates dictionary of results
		#Dictionary key is category
        self.pq_per_cat = defaultdict(PQStatCat)

    def __getitem__(self, i):
        return self.pq_per_cat[i]

    def __iadd__(self, pq_stat):
        for label, pq_stat_cat in pq_stat.pq_per_cat.items():
            self.pq_per_cat[label] += pq_stat_cat
        return self

    def pq_average(self, categories, isthing):
        pq, sq, rq, n = 0, 0, 0, 0
        per_class_results = {}
        for label, label_info in categories.items():
            if isthing is not None:
                cat_isthing = label_info['isthing'] == 1
                if isthing != cat_isthing:
                    continue
            iou = self.pq_per_cat[label].iou
            tp = self.pq_per_cat[label].tp
            fp = self.pq_per_cat[label].fp
            fn = self.pq_per_cat[label].fn
            if tp + fp + fn == 0:
                per_class_results[label] = {'pq': 0.0, 'sq': 0.0, 'rq': 0.0}
                continue
            n += 1
            pq_class = iou / (tp + 0.5 * fp + 0.5 * fn)
            sq_class = iou / tp if tp != 0 else 0
            rq_class = tp / (tp + 0.5 * fp + 0.5 * fn)
            per_class_results[label] = {'pq': pq_class, 'sq': sq_class, 'rq': rq_class}
            pq += pq_class
            sq += sq_class
            rq += rq_class

        return {'pq': pq / n, 'sq': sq / n, 'rq': rq / n, 'n': n}, per_class_results

In [3]:
@get_traceback
def pq_compute_single_core(proc_id, annotation_set, gt_folder, pred_folder, categories):
    pq_stat = PQStat()
    
    print("pq_compute running...")
    
    idx = 0
    for gt_ann, pred_ann in annotation_set:
        if idx % 100 == 0:
            print('Core: {}, {} from {} images processed'.format(proc_id, idx, len(annotation_set)))
        idx += 1

        pan_gt = np.array(Image.open(os.path.join(gt_folder, gt_ann['file_name'])), dtype=np.uint32)
        pan_gt = rgb2id(pan_gt)
        pan_pred = np.array(Image.open(os.path.join(pred_folder, pred_ann['file_name'])), dtype=np.uint32)
        pan_pred = rgb2id(pan_pred)

        gt_segms = {el['id']: el for el in gt_ann['segments_info']}
        pred_segms = {el['id']: el for el in pred_ann['segments_info']}

        # predicted segments area calculation + prediction sanity checks
        pred_labels_set = set(el['id'] for el in pred_ann['segments_info'])
        labels, labels_cnt = np.unique(pan_pred, return_counts=True)
        for label, label_cnt in zip(labels, labels_cnt):
            if label not in pred_segms:
                if label == VOID:
                    continue
                raise KeyError('In the image with ID {} segment with ID {} is presented in PNG and not presented in JSON.'.format(gt_ann['image_id'], label))
            pred_segms[label]['area'] = label_cnt
            pred_labels_set.remove(label)
            if pred_segms[label]['category_id'] not in categories:
                raise KeyError('In the image with ID {} segment with ID {} has unknown category_id {}.'.format(gt_ann['image_id'], label, pred_segms[label]['category_id']))
        if len(pred_labels_set) != 0:
            raise KeyError('In the image with ID {} the following segment IDs {} are presented in JSON and not presented in PNG.'.format(gt_ann['image_id'], list(pred_labels_set)))

        # confusion matrix calculation
        pan_gt_pred = pan_gt.astype(np.uint64) * OFFSET + pan_pred.astype(np.uint64)
        gt_pred_map = {}
        labels, labels_cnt = np.unique(pan_gt_pred, return_counts=True)
        for label, intersection in zip(labels, labels_cnt):
            gt_id = label // OFFSET
            pred_id = label % OFFSET
            gt_pred_map[(gt_id, pred_id)] = intersection

        # count all matched pairs
        gt_matched = set()
        pred_matched = set()
        for label_tuple, intersection in gt_pred_map.items():
            gt_label, pred_label = label_tuple
            if gt_label not in gt_segms:
                continue
            if pred_label not in pred_segms:
                continue
            if gt_segms[gt_label]['iscrowd'] == 1:
                continue
            if gt_segms[gt_label]['category_id'] != pred_segms[pred_label]['category_id']:
                continue

            union = pred_segms[pred_label]['area'] + gt_segms[gt_label]['area'] - intersection - gt_pred_map.get((VOID, pred_label), 0)
            iou = intersection / union
            if iou > 0.5:
                pq_stat[gt_segms[gt_label]['category_id']].tp += 1
                pq_stat[gt_segms[gt_label]['category_id']].iou += iou
                gt_matched.add(gt_label)
                pred_matched.add(pred_label)

        # count false positives
        crowd_labels_dict = {}
        for gt_label, gt_info in gt_segms.items():
            if gt_label in gt_matched:
                continue
            # crowd segments are ignored
            if gt_info['iscrowd'] == 1:
                crowd_labels_dict[gt_info['category_id']] = gt_label
                continue
            pq_stat[gt_info['category_id']].fn += 1

        # count false positives
        for pred_label, pred_info in pred_segms.items():
            if pred_label in pred_matched:
                continue
            # intersection of the segment with VOID
            intersection = gt_pred_map.get((VOID, pred_label), 0)
            # plus intersection with corresponding CROWD region if it exists
            if pred_info['category_id'] in crowd_labels_dict:
                intersection += gt_pred_map.get((crowd_labels_dict[pred_info['category_id']], pred_label), 0)
            # predicted segment is ignored if more than half of the segment correspond to VOID and CROWD regions
            if intersection / pred_info['area'] > 0.5:
                continue
            pq_stat[pred_info['category_id']].fp += 1
    print('Core: {}, all {} images processed'.format(proc_id, len(annotation_set)))
    return pq_stat

In [4]:
def pq_compute(gt_json_file, pred_json_file, gt_folder=None, pred_folder=None):

    start_time = time.time()
    with open(gt_json_file, 'r') as f:
        gt_json = json.load(f)
    with open(pred_json_file, 'r') as f:
        pred_json = json.load(f)

    if gt_folder is None:
        gt_folder = gt_json_file.replace('.json', '')
    if pred_folder is None:
        pred_folder = pred_json_file.replace('.json', '')
    categories = {el['id']: el for el in gt_json['categories']}

    print("Evaluation panoptic segmentation metrics:")
    print("Ground truth:")
    print("\tSegmentation folder: {}".format(gt_folder))
    print("\tJSON file: {}".format(gt_json_file))
    print("Prediction:")
    print("\tSegmentation folder: {}".format(pred_folder))
    print("\tJSON file: {}".format(pred_json_file))

    if not os.path.isdir(gt_folder):
        raise Exception("Folder {} with ground truth segmentations doesn't exist".format(gt_folder))
    if not os.path.isdir(pred_folder):
        raise Exception("Folder {} with predicted segmentations doesn't exist".format(pred_folder))

    pred_annotations = {el['image_id']: el for el in pred_json['annotations']}
    matched_annotations_list = []
    for gt_ann in gt_json['annotations']:
        image_id = gt_ann['image_id']
        if image_id not in pred_annotations:
            raise Exception('no prediction for the image with id: {}'.format(image_id))
        matched_annotations_list.append((gt_ann, pred_annotations[image_id]))

    cpu_num = multiprocessing.cpu_count()
    annotations_split = np.array_split(matched_annotations_list, cpu_num)
    print("Number of cores: {}, images per core: {}".format(cpu_num, len(annotations_split[0])))
    workers = multiprocessing.Pool(processes=cpu_num)
    processes = []
    for proc_id, annotation_set in enumerate(annotations_split):
        
        print("proc_id =", proc_id)
        
        p = workers.apply_async(pq_compute_single_core,
                                (proc_id, annotation_set, gt_folder, pred_folder, categories))
        processes.append(p)
    pq_stat = PQStat()
    for p in processes:
        pq_stat += p.get()

    metrics = [("All", None), ("Things", True), ("Stuff", False)]
    results = {}
    for name, isthing in metrics:
        results[name], per_class_results = pq_stat.pq_average(categories, isthing=isthing)
        if name == 'All':
            results['per_class'] = per_class_results
    print("{:10s}| {:>5s}  {:>5s}  {:>5s} {:>5s}".format("", "PQ", "SQ", "RQ", "N"))
    print("-" * (10 + 7 * 4))

    for name, _isthing in metrics:
        print("{:10s}| {:5.1f}  {:5.1f}  {:5.1f} {:5d}".format(
              name, 100 * results[name]['pq'], 100 * results[name]['sq'], 100 * results[name]['rq'], results[name]['n']))

    t_delta = time.time() - start_time
    print("Time elapsed: {:0.2f} seconds".format(t_delta))

    return results

In [159]:
gt_dir = r'C:\Users\andor.neo\Desktop\casey\ml_ai\models-master\research\deeplab\datasets\coco\annotations\\'
pred_dir = r'C:\Users\andor.neo\Desktop\casey\ml_ai\models-master\research\deeplab\datasets\coco\annotations\\'
gt_json_file = gt_dir + 'panoptic_test-dev2017_mrcnn.json'
pred_json_file = gt_dir + 'panoptic_test-dev2017_mrcnn.json'
gt_folder = gt_dir + 'panoptic_annotations_test2017'
pred_folder = gt_dir + 'panoptic_annotations_test2017'

In [160]:
with open(gt_json_file, 'r') as f:
    gt_json = json.load(f)
with open(pred_json_file, 'r') as f:
    pred_json = json.load(f)

In [161]:
if gt_folder is None:
    gt_folder = gt_json_file.replace('.json', '')
if pred_folder is None:
    pred_folder = pred_json_file.replace('.json', '')
categories = {el['id']: el for el in gt_json['categories']}

In [162]:
pred_annotations = {el['image_id']: el for el in pred_json['annotations']}
matched_annotations_list = []
for gt_ann in gt_json['annotations']:
    image_id = gt_ann['image_id']
    if image_id not in pred_annotations:
        raise Exception('no prediction for the image with id: {}'.format(image_id))
    matched_annotations_list.append((gt_ann, pred_annotations[image_id]))

In [163]:
cpu_num = multiprocessing.cpu_count()
annotations_split = np.array_split(matched_annotations_list, cpu_num)

In [164]:
#Adapted to catch and log errors for debugging
VOID=0
bad_image_ids = []
bad_segment_ids = {}
errors = {}

for annotation_set in annotations_split:
    idx = 0
    for gt_ann, pred_ann in annotation_set:
        if idx % 100 == 0:
            print(idx)
        idx += 1
        try:
            pan_gt = np.array(Image.open(os.path.join(gt_folder, gt_ann['file_name'])), dtype=np.uint32)
            pan_gt = rgb2id(pan_gt)
            pan_pred = np.array(Image.open(os.path.join(pred_folder, pred_ann['file_name'])), dtype=np.uint32)
            pan_pred = rgb2id(pan_pred)

            gt_segms = {el['id']: el for el in gt_ann['segments_info']}
            pred_segms = {el['id']: el for el in pred_ann['segments_info']}

            # predicted segments area calculation + prediction sanity checks
            pred_labels_set = set(el['id'] for el in pred_ann['segments_info'])
            labels, labels_cnt = np.unique(pan_pred, return_counts=True)
            for label, label_cnt in zip(labels, labels_cnt):
                if label not in pred_segms:
                    if label == VOID:
                        continue
                    raise KeyError('In the image with ID {} segment with ID {} is presented in PNG and not presented in JSON.'.format(gt_ann['image_id'], label))
                pred_segms[label]['area'] = label_cnt
                pred_labels_set.remove(label)
                if pred_segms[label]['category_id'] not in categories:
                    raise KeyError('In the image with ID {} segment with ID {} has unknown category_id {}.'.format(gt_ann['image_id'], label, pred_segms[label]['category_id']))
            if len(pred_labels_set) != 0:
                raise KeyError('In the image with ID {} the following segment IDs {} are presented in JSON and not presented in PNG.'.format(gt_ann['image_id'], list(pred_labels_set)))
        except KeyError as error:
            bad_image_ids.append(gt_ann['image_id'])
            bad_segment_ids[gt_ann['image_id']] = list(pred_labels_set)
            errors[gt_ann['image_id']] = error
            continue

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500


In [165]:
not_in_png = []
not_in_json = []
other = []
for image in bad_image_ids:
    if "not presented in PNG" in str(errors[image]):
        not_in_png.append(image)
    elif "not presented in JSON" in str(errors[image]):
        not_in_json.append(image)
    else:
        other.append(image)
print(len(not_in_png), len(not_in_json), len(other))

0 0 0


In [154]:
def default(o):
    if isinstance(o, np.int64): return int(o)  
    raise TypeError

In [177]:
with open(pred_json_file, "w") as fp:
    json.dump(pred_json, fp, default=default)

In [168]:
for image_id in not_in_json:
    
    for el in range(len(pred_json["annotations"])):
        
        if pred_json["annotations"][el]["image_id"] == image_id:
            idx = el
            
            err_dict= {pred_json["annotations"][idx]["segments_info"][el]["id"]:el for el in range(len(pred_json["annotations"][idx]["segments_info"]))}
            print(idx, err_dict)
            print(bad_segment_ids[image_id])

3832 {3937500: 0, 2363344: 1, 4522198: 2, 5112565: 3, 5383140: 4, 2369780: 5, 4851691: 6}


KeyError: 491864

In [94]:
#pred_json["annotations"][3832]["segments_info"] = new3832_anns
#pred_json["annotations"][3832]["segments_info"] == new3832_anns

True

In [118]:
#del pred_json["annotations"][3832]["segments_info"]
#pred_json["annotations"][3832]

{'image_id': 491864, 'file_name': '000000491864.png'}

In [70]:
#original_pred_json = original_pred_json.copy()

In [79]:
#Removes entries contained in JSON but not in PNG
for image_id in not_in_png:
    
    for el in range(len(pred_json["annotations"])):
        
        if pred_json["annotations"][el]["image_id"] == image_id:
            idx = el
            
            err_dict= {pred_json["annotations"][idx]["segments_info"][el]["id"]:el for el in range(len(pred_json["annotations"][idx]["segments_info"]))}
            #print(idx, err_dict)
    
    remove_these_idxs = []
    for bad_segment in bad_segment_ids[image_id]:
        remove_these_idxs.append(err_dict[bad_segment])
    #print(bad_segment_ids[image_id], remove_these_idxs)
    
    remove_these_idxs.sort(reverse=True)
    
    for remove_idx in remove_these_idxs:
        #del pred_json["annotations"][idx]["segments_info"][remove_idx]
        print(remove_idx, pred_json["annotations"][idx]["segments_info"][remove_idx])

KeyError: 44228

In [171]:
with open(r"C:\Users\andor.neo\Desktop\casey\ml_ai\panopticapi-master\panoptic_coco_categories.json", "r") as fp:
    panapi_cats = json.load(fp)
panapi_cats

[{'supercategory': 'person',
  'color': [220, 20, 60],
  'isthing': 1,
  'id': 1,
  'name': 'person'},
 {'supercategory': 'vehicle',
  'color': [119, 11, 32],
  'isthing': 1,
  'id': 2,
  'name': 'bicycle'},
 {'supercategory': 'vehicle',
  'color': [0, 0, 142],
  'isthing': 1,
  'id': 3,
  'name': 'car'},
 {'supercategory': 'vehicle',
  'color': [0, 0, 230],
  'isthing': 1,
  'id': 4,
  'name': 'motorcycle'},
 {'supercategory': 'vehicle',
  'color': [106, 0, 228],
  'isthing': 1,
  'id': 5,
  'name': 'airplane'},
 {'supercategory': 'vehicle',
  'color': [0, 60, 100],
  'isthing': 1,
  'id': 6,
  'name': 'bus'},
 {'supercategory': 'vehicle',
  'color': [0, 80, 100],
  'isthing': 1,
  'id': 7,
  'name': 'train'},
 {'supercategory': 'vehicle',
  'color': [0, 0, 70],
  'isthing': 1,
  'id': 8,
  'name': 'truck'},
 {'supercategory': 'vehicle',
  'color': [0, 0, 192],
  'isthing': 1,
  'id': 9,
  'name': 'boat'},
 {'supercategory': 'outdoor',
  'color': [250, 170, 30],
  'isthing': 1,
  'id'

In [172]:
panapi_cats[0]

{'supercategory': 'person',
 'color': [220, 20, 60],
 'isthing': 1,
 'id': 1,
 'name': 'person'}

In [174]:
#del pred_json["categories"]
#pred_json["categories"]

KeyError: 'categories'

In [176]:
#pred_json["categories"] = panapi_cats
pred_json["categories"][0]

{'supercategory': 'person',
 'color': [220, 20, 60],
 'isthing': 1,
 'id': 1,
 'name': 'person'}